In [ ]:
import pandas as pd
df = pd.read_csv('./NS_main_mid_sub1_cat.csv')
print(len(df))
df.head()

In [ ]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

ua = UserAgent(verify_ssl=False) # inspried by Nara's code
fake_ua = ua.random
headers = { 'user-agent' : fake_ua }

url = 'https://search.shopping.naver.com/search/category/{}'.format('50003086')
res = requests.get(url, headers=headers)

soup = BeautifulSoup(res.text, 'html.parser')
# soup.select('.style_content__xWg5l .noResult_no_result__6kgzY')

mid_cat_list = soup.select('.filter_finder_list__4PE1C .filter_text_over__iesoO')
mid_cat_list

# mid_cat_list = [mid_cat.text for mid_cat in mid_cat_list] # 

In [ ]:
# 눌렀는데 없는 거면 pass
# 1. 눌렀는데 있는 건데 동일하면 pass
# 2. 동일하지 않으면 긁어옴
# 3. 여기서 만약에 들어갔는데 리스트로로 또 뽑았는데 없으면 => 검색결과가 없습니다
#        있으면 검색결과가 있는 것이라고 확인해주는 건 있으면 좋을 것 같음. 

In [ ]:
# why this process?
# => 하려고하는 메인 카테고리를 정하고 그에 따른 카테고리 이름, 아이디를 추출하는 데이터 프레임을 뽑기 위해
# => 스마트 스토어 접근이 어려웠다면? 
import re
from bs4 import BeautifulSoup
import time
from collections import defaultdict
from selenium import webdriver # webdriver를 이용해 해당 브라우저 열기
from selenium.webdriver import ActionChains # 일련의 작업들을 연속적으로 진행할 수 있도록 도와줌
from selenium.webdriver.common.keys import Keys # 키보드 입력을 할 수 있게 하기 위해
from selenium.webdriver.common.by import By # html요소 탐색을 할 수 있게 하기 위해
from selenium.webdriver.support.ui import WebDriverWait # 브라우저 응답을 기다릴 수 있게 하기 위해
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC # html 요소의 상태를 체크할 수 있게 하기 위해
import pandas as pd

# -- 처음 데이터 프레임 -- 
df = pd.read_csv('./NS_main_mid_sub1_cat.csv') 

def make_dict_by_mainCat(mainCat: str) -> dict:
    """
    mainCat별 midCat과 sub1Cat 리스트로 나누어주는 
    딕션어리를 만들어주는 함수입니다. 
    """
    main_mid_sub1_dict = {}
    
    # -- mainCat 기준으로 데이터 프레임 재생성 -- 
    df_by_main = df[df['mainCat_name'] == mainCat]
    
    # -- 데이터 프레임에서 unique한 midCat 구하기 --
    midCat_list = list(df_by_main['midCat_name'].unique())
    
    # -- midCat에 따라 sub1Cat 뽑아와서 dictionary 형태로 만들기 --
    mid_sub1_dict = defaultdict(list)
    for midCat in midCat_list:
        sub1_list = list(df_by_main[df_by_main['midCat_name'] == midCat]['sub1Cat_name'].unique()) # midCat에 따른 unique한 sub1 리스트
        mid_sub1_dict[midCat] = sub1_list
        
    # -- mid_sub1_dict와 mainCat합쳐서 새로운 딕션어리 생성 --
    main_mid_sub1_dict[mainCat] = dict(mid_sub1_dict)
    
    return main_mid_sub1_dict

def bring_sub2Cat_id_from_url():
    """
    url에서 sub2Cat_id만 가져오는 함수
    """
    current_url = dr.current_url # 현재 주소에서 가져옴
    cat_id = re.findall(r'\/\w+\?', current_url)[0].lstrip('/').rstrip('?') 
    return cat_id

def click_by_cat(cat):
    """
    category별로 클릭해주는 함수
    """
    el = dr.find_element_by_link_text(cat) 
    el.click()
    ### 여기도 clickable 하게 만드는 코드 ###
    return 

def bring_sub2Cat_Series(mainCat_dict: dict): # 대분류 딕션어리 하나 받았다고 가정하고
    
    sub2Cat_list = list()       #### 여기 바뀌어야되 ####
    for mainCat, midCat_dict in mainCat_dict.items(): # mainCat & midCat list
        for midCat, sub1Cat_list in midCat_dict.items(): # midCat & sub1Cat list
            for sub1Cat in sub1Cat_list: # sub1Cat
                
                # -- 다음 sub2Cat 리스트 뽑아내기 --
                url = 'https://search.shopping.naver.com/search/category/{}'.format(sub1Cat) # 처음부터 sub1Cat따라 url 들어오기
                dr.get(url)
                
                # -- class 부분이 클릭이 가능해질 때까지 wait --
                wait = WebDriverWait(driver, 8)
                element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'filter_finder_list__4PE1C')))
                
                # -- beatifulsoup으로 sub2에 해당하는 리스트 가져오기 -- 
                soup = BeautifulSoup(dr.page_source, 'html.parser')
                sub2_list = soup.select('.filter_finder_list__4PE1C .filter_text_over__iesoO')
                sub2_list = [sub2.text for sub2 in sub2_list]
                
                if (sub2_list == sub1Cat_list) | (sub2_list == []):
                    sub2Cat_list.append([sub1Cat, ' ', ' ']) # sub2가 없는 경우 빈칸과 '_' 조인
                    
                else:    
                    for sub2Cat_name in sub2_list:
                        # -- sub2Cat 클릭 --
                        click_by_cat(sub2Cat)
                        # -- 잠시 대기 --
                        dr.implicitly_wait(3)
                        # -- url에서 카테고리 아이디 가져오기 --
                        sub2Cat_id = bring_sub2Cat_id_from_url()
                        sub2Cat_list.append([sub1Cat, sub2Cat_id, sub2Cat_name]) # sub2 있는 경우 '_' 조인
                        # -- 다시 뒤로 돌아가기 -- 
                        click_by_cat(sub1Cat)
                        # -- 잠기 대기 --
                        dr.implicitly_wait(3)
                
                dr.close() # 다음 카테고리로 넘어가기 위해 창 닫아주기
                time.sleep(0.07)
    sub2Cat_df = pd.DataFrame(sub2Cat_list, columns=['sub1Cat_name', 'sub2Cat_id', 'sub2Cat_name'])
    return sub2Cat_df  # 기존 데이터 프레임에 merge시키기 위해 시리즈 데이터로 변경 
    
def merge_mainCat_sub2Cat_df(mainCat):
    mainCat_dict = make_dict_by_mainCat(mainCat)
    sub2Cat_df = bring_sub2Cat_Series(mainCat_dict)
    df_by_main = df[df['mainCat_name'] == mainCat]
    final_df = pd.merge(df_by_main, sub2Cat_df, left_on='sub1Cat_name', right_on='sub1Cat_name', how='left')
    
    return final_df # sub2Cat_name, sub2Cat_id까지 합쳐진 mainCat 별 데이터 프레임


# 1. series 데이터 '_'로 분리시켜서 데이터 프레임으로 변경뒤
# 2. 기존 데이터 프레임에 left merge 시켜주기 



In [ ]:
# 심지어 없는 부분에 대해서도 알아서 Nan 처리를 해줌 => 거의 다 왔다 조금만..!
pd.merge(tmp_df, tmp_df2, left_on='sub1Cat_name', right_on = 0, how='left')

In [113]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import json

ua = UserAgent(verify_ssl=False) 
fake_ua = ua.random
headers = {
    'user-agent' : fake_ua,
    'cache-control' : 'no-cache, no-store, must-revalidate',
    'sbth' : 'ea4aa25c9cf200dfd70e5669b535d2cbe252e2bca8ec494ef206a4da2f2cf2856382c6a40bba7f6cf1d067d37ebe2b10',
    'referer' : 'https://search.shopping.naver.com/search/category/100005587?',    
}

params = (
    ('sort','rel'),
    ('pagingIndex','2'),
    ('pagingSize','40'),
    ('viewType','list'),
    ('catId','50000790'),
)

url = 'https://search.shopping.naver.com/api/search/category'
res = requests.get(url, headers=headers, params=params )
data = res.json()
data['shoppingResult']['products']



[{'collection': 'product',
  'purchaseConditionInfos': [{'seq': '',
    'condition': '',
    'count': '',
    'lowPrice': '',
    'deliveryLowPrice': '',
    'unitPrice': '',
    'unit': '',
    'deliveryUnitPrice': '',
    'crUrl': 'https://cr.shopping.naver.com/adcr.nhn?x=nTeEfusCVPwKVOqKtFCheP%2F%2F%2Fw%3D%3Ds2GN9rHadsH8oNVZASmd51smzFy%2FdxHh4j9xXuJFpfp0dz4MSw%2BtTyncFUN4%2BVRJH5lktlpthRja11%2FZBaNrAekPg%2F6PYJW9LpCSumzPJoijqwaShusc4uN1D0982lqsAQCuK3pWhdO5xDKhpR805FHkz94uIkf%2FWNGGbMIynX4zfXvHC2ZqJ9WJ9Un3u29cze%2F1wwdwKClZ4rb32ZMynZzaSZny%2B1MgSfJG5ocTRQNj1rIvubP2%2F%2BNF2sTRY2hEV%2FNOLVor8UZ%2FlFrQ%2F0UiKOQPAAsG8Nwd%2FUGupBSq1ZAhO7WV%2BiipkvoAF85sVYpnJq4pZA3hvtay1NdVGnhX93YrHsbeStsmJBfOvK7DA6ok9tIrP9lQTXzSnut%2FVHrMeLmdNXoRCHjtF5tFJVvBe6uiCaLIlKOQKgEBHlY%2B0O0IH9qVndnr7Yr%2BqxLaTi%2BW2exTlulaZIxpt8%2FrQ4UYxIQRzfMyyBH7euWZyi2ibdpH3pGYdSOlpJBT60vjNpHZ%2B2riQX2bfy3CgLF0UFAzSXT%2FgXkjdQvSc3HXi7BfpfB2i0RWTorInxov1X47JoAzXVJyKhzL7xWcEX0DN0Ifk%2Fc2837KQ2EhoZ5Hya04J5fb7FIiSUS33x3YxDAjYvdMh